## Naive Bayes

### References:
* http://www.analyticsvidhya.com/blog/2015/09/naive-bayes-explained/
* https://egret.psychol.cam.ac.uk/statistics/local_copies_of_sources_Cardinal_and_Aitken_ANOVA/Bayes_theorem.htm#Example_.231:_False_positives_in_a_medical_test


#### Some Theory
Bayes Theorem

$$ P(X|Y) = \frac{P( X \cap Y)}{P(Y)} = \frac{P(Y|X) P(X)}{\sum_x{P(Y|x)} } $$



In [262]:
import sys
import importlib as imp
if ('Jupytils' in sys.modules):
    reloaded = imp.reload(Jupytils)
else:
    import Jupytils
    

In [263]:
fileName="../data/pima-indians-diabetes.csv"

df  = LoadDataSet(fileName);
displayDFs([df], maxrows=12)


count,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000
mean,3.845,120.895,69.105,20.536,79.799,31.993,0.472,33.241,0.349
std,3.370,31.973,19.356,15.952,115.244,7.884,0.331,11.760,0.477
min,0.000,0.000,0.000,0.000,0.000,0.000,0.078,21.000,0.000
25%,1.000,99.000,62.000,0.000,0.000,27.300,0.244,24.000,0.000
50%,3.000,117.000,72.000,23.000,30.500,32.000,0.372,29.000,0.000
75%,6.000,140.250,80.000,32.000,127.250,36.600,0.626,41.000,1.000
max,17.000,199.000,122.000,99.000,846.000,67.100,2.420,81.000,1.000
,,,,,,,,,
,Num_Pregnant (int64),Glucose_c (int64),BP (int64),SkinThick (int64),Serum_Insulin (int64),BMI (float64),Diabetes (float64),Age (int64),Diabetic (int64)
0,6,148,72,35,0,33.600,0.627,50,1


In [269]:
# Example of Naive Bayes implemented from Scratch in Python
import csv
import random
import math

def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    dataset = dataset[3:]
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	testSet = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(testSet))
		trainSet.append(testSet.pop(index))
	return [trainSet, testSet]

def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

def summarize(dataset):
	#summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	summaries = [(np.array(a).mean(), np.array(a).std(), np.array(a).var()) for a in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    labels={}
    y = list(zip(*dataset))[-1]
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
        labels[classValue] = y.count(classValue)/len(y)
    return summaries, labels

def calculateProbability(x, mean, stdev, var=None):
    if (var is None):
        var = math.pow(stdev,2)
    exponent = math.exp(-(math.pow(x-mean,2)/(2*var)))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, labels, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = labels[classValue]
		for i in range(len(classSummaries)):
			mean, stdev, var = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev, var)
	return probabilities
			
def predict(summaries, labels, inputVector):
	probabilities = calculateClassProbabilities(summaries, labels, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

def getPredictions(summaries, labels, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, labels, testSet[i])
		predictions.append(result)
	return predictions

def getProbs(summaries, labels, testSet):
    r = []
    for i in range(len(testSet)):
        p = calculateClassProbabilities(summaries, labels, testSet[i])
        r.append(p)
    return r


def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

## - Use of locally implemented Naive-Bayes
## 
filename = '../data/pima-indians-diabetes.csv'
splitRatio = 0.67
dataset = loadCsv(filename)
trainingSet, testSet = splitDataset(dataset, splitRatio)
print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)) )
# prepare model
summaries, labels = summarizeByClass(trainingSet)
# test model

#testSet = testSet[0:50]
predictions = getPredictions(summaries, labels, testSet)

probs = getProbs(summaries,labels, testSet)
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

Split 768 rows into train=514 and test=254 rows
Accuracy: 77.55905511811024%


### Using Python tools

In [270]:
def SplitXy(Xy):
    Xy10=Xy[0:8]
    Xy10 = Xy;
    #print Xy10
    #print "========"
    zXy10=list(zip(*Xy10))
    y= zXy10[-1]
    del zXy10[-1]
    z1=zip(*zXy10)
    X=[list(t) for t in z1]
    return np.array(X), np.array(y)

from sklearn.naive_bayes import GaussianNB
X,y = SplitXy(trainingSet) 
Xt,yt = SplitXy(testSet)

model = GaussianNB()
model.fit(X, y)


GaussianNB()

In [266]:
### Compare the models built by Python

print ("Class: 0")
for i,j in enumerate(model.theta_[0]):
    print ("({:8.2f} {:9.2f} {:9.2f} )".format(j, sqrt(model.sigma_[0][i]), model.sigma_[0][i]) , end="")
    print ("==> ", summaries[0][i])
    
print ("Class: 1")
for i,j in enumerate(model.theta_[1]):
    print ("({:8.2f} {:9.2f} {:9.2f} )".format(j, sqrt(model.sigma_[1][i]), model.sigma_[1][i]) , end="")
    print ("==> ", summaries[1][i])

Class: 0
(    3.17      2.95      8.70 )==>  (3.1722054380664653, 2.9493121170600949, 8.6984419638374977)
(  110.57     27.97    782.43 )==>  (110.56797583081571, 27.971890321689028, 782.4266481686002)
(   68.75     17.40    302.76 )==>  (68.74622356495469, 17.399923844283823, 302.75734978687672)
(   19.77     14.71    216.37 )==>  (19.773413897280967, 14.709678467293783, 216.37464061116637)
(   73.25    109.02  11885.31 )==>  (73.25377643504531, 109.01978371390005, 11885.313241025548)
(   30.47      7.45     55.51 )==>  (30.470996978851968, 7.4503475396941399, 55.507678462226529)
(    0.45      0.32      0.10 )==>  (0.44841993957703935, 0.31612071114257823, 0.099932304013289397)
(   31.18     12.08    145.97 )==>  (31.17522658610272, 12.081775351475576, 145.96929564352277)
Class: 1
(    4.87      3.59     12.87 )==>  (4.8743169398907105, 3.5881630608035673, 12.874914150915224)
(  141.36     32.28   1042.17 )==>  (141.35519125683061, 32.28272580838177, 1042.1743856191586)
(   70.30    

In [71]:
Pt = model.predict_proba(Xt)
Pt[0:10]

array([[  3.11e-01,   6.89e-01],
       [  9.77e-01,   2.33e-02],
       [  1.17e-03,   9.99e-01],
       [  9.72e-01,   2.83e-02],
       [  1.07e-05,   1.00e+00],
       [  8.38e-01,   1.62e-01],
       [  3.51e-01,   6.49e-01],
       [  1.25e-02,   9.87e-01],
       [  3.00e-01,   7.00e-01],
       [  1.32e-01,   8.68e-01]])

In [72]:
p1 = model.predict(Xt)
for i in range(len(predictions)):
    if (not predictions[i] == p1[i] ):
        print ("{} differ {} {}, {} ".format(i, predictions[i] , p1[i] , Xt[i]), end="\n")
        break;

In [53]:
probs[0:10]

[{0.0: 1.5144196669021058e-13, 1.0: 3.3504888419796258e-13},
 {0.0: 2.1659920128183192e-12, 1.0: 5.1652046600474917e-14},
 {0.0: 2.5252508722759389e-22, 1.0: 2.1583967093423703e-19},
 {0.0: 1.1955880288636698e-12, 1.0: 3.4878224094673554e-14},
 {0.0: 5.1180759947852162e-22, 1.0: 4.8008227794786382e-17},
 {0.0: 5.4594611576568794e-13, 1.0: 1.0549012601765871e-13},
 {0.0: 7.5881279382433877e-14, 1.0: 1.404839705685913e-13},
 {0.0: 4.9552646771852525e-16, 1.0: 3.9093638146880608e-14},
 {0.0: 9.9023505970477623e-14, 1.0: 2.3148834481099243e-13},
 {0.0: 4.0549065587723969e-16, 1.0: 2.6776226258262385e-15}]

In [54]:
X,y = SplitXy(trainingSet)
yy=np.array(y)
y0=yy[yy==0]
y1=yy[yy==1]
p0=len(y0)/len(y)
p1=len(y1)/len(y)
print(len(y0), len(y1), len(y), p0, p1)

329 185 514 0.6400778210116731 0.35992217898832685


In [73]:
from dill.source import getsource
print(getsource(model.predict_log_proba))

    def predict_log_proba(self, X):
        """
        Return log-probability estimates for the test vector X.

        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]

        Returns
        -------
        C : array-like, shape = [n_samples, n_classes]
            Returns the log-probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute `classes_`.
        """
        jll = self._joint_log_likelihood(X)
        # normalize by P(x) = P(f_1, ..., f_n)
        log_prob_x = logsumexp(jll, axis=1)
        return jll - np.atleast_2d(log_prob_x).T



In [57]:
print(getsource(model._joint_log_likelihood))

    def _joint_log_likelihood(self, X):
        check_is_fitted(self, "classes_")

        X = check_array(X)
        joint_log_likelihood = []
        for i in range(np.size(self.classes_)):
            jointi = np.log(self.class_prior_[i])
            n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
            n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) /
                                 (self.sigma_[i, :]), 1)
            joint_log_likelihood.append(jointi + n_ij)

        joint_log_likelihood = np.array(joint_log_likelihood).T
        return joint_log_likelihood



In [335]:
print(model.class_prior_[0], model.class_prior_[1])

0.655642023346 0.344357976654


In [282]:
jll = model._joint_log_likelihood(Xt)
#jll

In [82]:
#Import Library of Gaussian Naive Bayes model

#assigning predictor and target variables

#Output: ([3,4])


[3 4]


In [99]:
x.mean(axis=0)
x.var(axis=0)

array([ 4.58,  7.  ])

In [92]:
x1 = x[y==3]


In [338]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

class myNB:
    def __init__(self):
        pass;
    
    def fit(self,x,y):
        if (type(y) == tuple):
            y = np.array(y)
        labels = unique(y)
        params = defaultdict(dict)
        for k in (labels):
            x1 = x[y==k]
            params[k]['prior'] = len(x1)/len(x)
            params[k]['mean'] = x1.mean(axis=0);
            params[k]['var' ] = x1.var(axis=0);
            
        self.labels = labels
        self.params = params
    
    def Gaussian(self, x, mean, var):
        e = math.exp(-(math.pow(x-mean,2)/(2*var)))
        return (1 / (math.sqrt(2*math.pi * var)) * e)

    # Predict product probability 
    def predict_proba1(self, x):
        p = {}
        for k, parms in self.params.items():
            p[k] = parms['prior']
            for i in range(len(parms['mean'])):
                mean, var = parms['mean'][i], parms['var'][i]
                p[k] *= self.Gaussian( x[i], mean, var)
        return p
    
    def predict_proba(self, Xt, dump=False):
        r = []
        for x in Xt:
            p = self.predict_proba1(x)
            w = max(p, key=p.get)
            p['Predicted'] = w
            r.append(p)
        
        if (dump):
            for i in r:
                print(i)
        return r;

    ## Predict Log probabilities
    def predict_log_proba1(self, x):
        p = {}
        for k, parms in self.params.items():
            p[k] = log(parms['prior'])
            for i in range(len(parms['mean'])):
                mean, var = parms['mean'][i], parms['var'][i]
                p[k] += log(self.Gaussian( x[i], mean, var))
        return p

    def predict_log_proba(self, Xt, dump=False):
        r = []
        for x in Xt:
            p = self.predict_log_proba1(x)
            w = max(p, key=p.get)
            p['Predicted'] = w
            r.append(p)
        
        if (dump):
            for i in r:
                print(i)
        return r;

    def predict(self, Xt, dump=False):
        r = predict_proba(Xt, dump)
        pass
    
    def dump(self):
        print("My NB : labels =>", self.labels);
        for k in self.labels:
            p = self.params[k]
            print("Class: ", k, "\n  Prior:" , p['prior'], "\n  mean:", p['mean'],"\n   var:", p['var']  )
            #print("{} ==> mean: {:8.2f}  Var: {:9.2f} )".format(j, self.params[k]) , end="")
        pass

XX = np.array([[-3,7],[1,5], [1,2], [-2,0], [2,3], [-4,0], [-1,1], [1,1], [-2,2], [2,7], [-4,1], [-2,7]])
yy = np.array([0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1])

myModel = myNB()

# Train the model using the training sets 
pyModel.fit(XX, yy)
                
myModel.fit(XX,yy)
myModel.dump()

predicted_prob= myModel.predict_proba([[1,2],[3,4]], True)

#pyModel = GaussianNB()
#print ("Gaussian NB parameters: ")
#for k in myModel.labels:
#    for i,j in enumerate(pyModel.theta_[k]):
        #print (k)
#        print ("{} ({:6.2f} {:6.2f} )".format(k, j, model.sigma_[k][i]) )
    
#Predict Output 
#predicted= model.predict([[1,2],[3,4]])
#print( predicted)

My NB : labels => [0 1]
Class:  0 
  Prior: 0.5833333333333334 
  mean: [-1.43  2.43] 
   var: [ 3.1   5.96]
Class:  1 
  Prior: 0.4166666666666667 
  mean: [-0.2  3.8] 
   var: [ 5.76  7.36]
{0: 0.008217910623911775, 1: 0.007212402228071897, 'Predicted': 0}
{0: 0.0007437940158082369, 1: 0.004175793880737992, 'Predicted': 1}


In [339]:
nb = myNB()
nb.fit(np.array(X),y)
nb.dump()

My NB : labels => [ 0.  1.]
Class:  0.0 
  Prior: 0.6556420233463035 
  mean: [   3.43  111.7    68.13   18.99   70.73   30.3     0.44   31.62] 
   var: [  9.79e+00   6.54e+02   3.29e+02   2.23e+02   1.09e+04   5.52e+01   8.95e-02   1.43e+02]
Class:  1.0 
  Prior: 0.3443579766536965 
  mean: [   5.07  141.66   70.95   21.67  102.81   35.06    0.52   37.88] 
   var: [  1.43e+01   9.31e+02   4.48e+02   3.19e+02   2.25e+04   4.27e+01   1.17e-01   1.25e+02]


In [318]:
### Compare three models built by Google-Link, Sada, Python Lib

for k in ([0,1]):
    print ("Class: ", k)
    p = nb.params[k];
    for i,j in enumerate(model.theta_[k]):
        print ("({:8.2f} {:9.2f} )".format(j, model.sigma_[k][i]) , end="")
        print ("==>({:8.2f} {:9.2f} )".format(summaries[k][i][0], summaries[k][i][2] ), end ="")
        print ("==>({:8.2f} {:9.2f} )".format(p['mean'][i], p['var'][i]))


Class:  0
(    3.43      9.79 )==>(    3.43      9.79 )==>(    3.43      9.79 )
(  111.70    654.49 )==>(  111.70    654.49 )==>(  111.70    654.49 )
(   68.13    329.07 )==>(   68.13    329.07 )==>(   68.13    329.07 )
(   18.99    222.55 )==>(   18.99    222.55 )==>(   18.99    222.55 )
(   70.73  10906.59 )==>(   70.73  10906.59 )==>(   70.73  10906.59 )
(   30.30     55.20 )==>(   30.30     55.20 )==>(   30.30     55.20 )
(    0.44      0.09 )==>(    0.44      0.09 )==>(    0.44      0.09 )
(   31.62    143.15 )==>(   31.62    143.15 )==>(   31.62    143.15 )
Class:  1
(    5.07     14.28 )==>(    5.07     14.28 )==>(    5.07     14.28 )
(  141.66    931.44 )==>(  141.66    931.44 )==>(  141.66    931.44 )
(   70.95    447.60 )==>(   70.95    447.60 )==>(   70.95    447.60 )
(   21.67    319.08 )==>(   21.67    319.08 )==>(   21.67    319.08 )
(  102.81  22523.10 )==>(  102.81  22523.10 )==>(  102.81  22523.10 )
(   35.06     42.65 )==>(   35.06     42.65 )==>(   35.06     42.65 )


In [319]:
probs[0:20]

[{0.0: 1.8105490310301792e-13, 1.0: 3.9635923552404768e-13},
 {0.0: 1.6017911588647289e-12, 1.0: 3.0058659740266122e-14},
 {0.0: 8.0099555301913151e-15, 1.0: 2.2423283322909947e-14},
 {0.0: 1.4055383582267548e-12, 1.0: 1.751120577489116e-14},
 {0.0: 1.2172910489803623e-12, 1.0: 6.2526182158064153e-14},
 {0.0: 1.2475447144106206e-14, 1.0: 1.2675705210115209e-13},
 {0.0: 9.9049665209287208e-16, 1.0: 4.0958959308544996e-16},
 {0.0: 1.432164938321917e-14, 1.0: 1.8297482185374715e-14},
 {0.0: 9.9662984049131054e-13, 1.0: 1.0011588795259298e-13},
 {0.0: 3.5596414848055093e-14, 1.0: 8.5278355615602826e-15},
 {0.0: 3.0366540148132589e-12, 1.0: 4.0069284408044781e-13},
 {0.0: 1.2333461433792027e-13, 1.0: 6.0633242379741584e-14},
 {0.0: 8.8944077295233952e-16, 1.0: 4.4377298569807841e-14},
 {0.0: 2.992756114209074e-13, 1.0: 2.5031075164434282e-13},
 {0.0: 5.6984160788679779e-14, 1.0: 1.6793138461726468e-13},
 {0.0: 2.7655299557307894e-13, 1.0: 3.2582941608865697e-14},
 {0.0: 1.5902403869405197e-

In [323]:
p1=nb.predict_proba(Xt)
p2=nb.predict_log_proba(Xt)
display(p1[0:20])
display(p2[0:20])

[{0.0: 1.8105490310301761e-13, 1.0: 3.963592355240475e-13, 'Predicted': 1.0},
 {0.0: 1.6017911588647285e-12, 1.0: 3.0058659740266154e-14, 'Predicted': 0.0},
 {0.0: 8.009955530191307e-15, 1.0: 2.242328332291001e-14, 'Predicted': 1.0},
 {0.0: 1.4055383582267558e-12, 1.0: 1.751120577489115e-14, 'Predicted': 0.0},
 {0.0: 1.2172910489803617e-12, 1.0: 6.252618215806427e-14, 'Predicted': 0.0},
 {0.0: 1.2475447144106209e-14, 1.0: 1.2675705210115207e-13, 'Predicted': 1.0},
 {0.0: 9.904966520928654e-16, 1.0: 4.0958959308545104e-16, 'Predicted': 0.0},
 {0.0: 1.4321649383219157e-14, 1.0: 1.8297482185374705e-14, 'Predicted': 1.0},
 {0.0: 9.966298404913101e-13, 1.0: 1.0011588795259305e-13, 'Predicted': 0.0},
 {0.0: 3.5596414848055074e-14, 1.0: 8.527835561560272e-15, 'Predicted': 0.0},
 {0.0: 3.036654014813258e-12, 1.0: 4.006928440804474e-13, 'Predicted': 0.0},
 {0.0: 1.2333461433792027e-13, 1.0: 6.06332423797416e-14, 'Predicted': 0.0},
 {0.0: 8.894407729523393e-16, 1.0: 4.4377298569807803e-14, 'Pred

[{0.0: -29.339976077579863, 1.0: -28.556455434463277, 'Predicted': 1.0},
 {0.0: -27.159898638536436, 1.0: -31.135625597732005, 'Predicted': 0.0},
 {0.0: -32.458091185632185, 1.0: -31.428676541828736, 'Predicted': 1.0},
 {0.0: -27.290600713419785, 1.0: -31.67593538891121, 'Predicted': 0.0},
 {0.0: -27.434393177696933, 1.0: -30.403191011359496, 'Predicted': 0.0},
 {0.0: -32.015013910700532, 1.0: -29.696504116108212, 'Predicted': 1.0},
 {0.0: -34.548325187777223, 1.0: -35.431376007996512, 'Predicted': 0.0},
 {0.0: -31.877004059618539, 1.0: -31.632012930027717, 'Predicted': 1.0},
 {0.0: -27.634396967216531, 1.0: -29.932448000379196, 'Predicted': 0.0},
 {0.0: -30.96653146863925, 1.0: -32.395440809854115, 'Predicted': 0.0},
 {0.0: -26.520264859695732, 1.0: -28.545581235974478, 'Predicted': 0.0},
 {0.0: -29.723875291470783, 1.0: -30.433933098115084, 'Predicted': 0.0},
 {0.0: -34.655938753655462, 1.0: -30.746048349700384, 'Predicted': 1.0},
 {0.0: -28.83741146876838, 1.0: -29.016073242364165, 

In [324]:
Pt = model.predict_proba(Xt)
Pt[0:20]

array([[ 0.31,  0.69],
       [ 0.98,  0.02],
       [ 0.26,  0.74],
       [ 0.99,  0.01],
       [ 0.95,  0.05],
       [ 0.09,  0.91],
       [ 0.71,  0.29],
       [ 0.44,  0.56],
       [ 0.91,  0.09],
       [ 0.81,  0.19],
       [ 0.88,  0.12],
       [ 0.67,  0.33],
       [ 0.02,  0.98],
       [ 0.54,  0.46],
       [ 0.25,  0.75],
       [ 0.89,  0.11],
       [ 0.57,  0.43],
       [ 0.91,  0.09],
       [ 0.37,  0.63],
       [ 0.  ,  1.  ]])

In [387]:
jll = model._joint_log_likelihood(Xt)
jll[0:20]

array([[-29.34, -28.56],
       [-27.16, -31.14],
       [-32.46, -31.43],
       [-27.29, -31.68],
       [-27.43, -30.4 ],
       [-32.02, -29.7 ],
       [-34.55, -35.43],
       [-31.88, -31.63],
       [-27.63, -29.93],
       [-30.97, -32.4 ],
       [-26.52, -28.55],
       [-29.72, -30.43],
       [-34.66, -30.75],
       [-28.84, -29.02],
       [-30.5 , -29.42],
       [-28.92, -31.05],
       [-29.47, -29.75],
       [-28.23, -30.58],
       [-29.75, -29.21],
       [-38.26, -32.92]])

In [382]:
r=sum(jll,1)
sum(jll,1)[:, np.newaxis]

array([[ -57.9 ],
       [ -58.3 ],
       [ -63.89],
       [ -58.97],
       [ -57.84],
       [ -61.71],
       [ -69.98],
       [ -63.51],
       [ -57.57],
       [ -63.36],
       [ -55.07],
       [ -60.16],
       [ -65.4 ],
       [ -57.85],
       [ -59.91],
       [ -59.97],
       [ -59.22],
       [ -58.81],
       [ -58.96],
       [ -71.18],
       [ -83.24],
       [ -60.17],
       [ -60.71],
       [ -65.42],
       [ -95.07],
       [ -63.65],
       [ -56.69],
       [ -56.9 ],
       [ -72.93],
       [ -60.86],
       [ -57.99],
       [ -65.63],
       [ -55.7 ],
       [ -66.44],
       [ -57.96],
       [ -65.39],
       [ -54.64],
       [ -63.23],
       [ -58.23],
       [ -59.08],
       [ -61.43],
       [ -58.44],
       [ -68.25],
       [ -55.55],
       [ -59.56],
       [ -58.13],
       [ -57.  ],
       [ -72.01],
       [ -55.93],
       [ -56.52],
       [ -64.79],
       [ -62.2 ],
       [ -65.02],
       [ -60.46],
       [ -63.78],
       [ -

In [357]:
log_prob_x = scipy.misc.logsumexp(jll, axis=1)
log_prob_x[0:20]

array([-28.18, -27.14, -31.12, -27.28, -27.38, -29.6 , -34.2 , -31.05, -27.54, -30.75, -26.4 , -29.32, -30.73, -28.23, -29.12, -28.8 , -28.91, -28.14, -28.75, -32.92])

In [361]:
ps= jll - np.atleast_2d(log_prob_x).T
ps[0:20]

array([[ -1.16e+00,  -3.76e-01],
       [ -1.86e-02,  -3.99e+00],
       [ -1.33e+00,  -3.05e-01],
       [ -1.24e-02,  -4.40e+00],
       [ -5.01e-02,  -3.02e+00],
       [ -2.41e+00,  -9.39e-02],
       [ -3.46e-01,  -1.23e+00],
       [ -8.23e-01,  -5.78e-01],
       [ -9.57e-02,  -2.39e+00],
       [ -2.15e-01,  -1.64e+00],
       [ -1.24e-01,  -2.15e+00],
       [ -4.00e-01,  -1.11e+00],
       [ -3.93e+00,  -1.98e-02],
       [ -6.08e-01,  -7.86e-01],
       [ -1.37e+00,  -2.92e-01],
       [ -1.11e-01,  -2.25e+00],
       [ -5.62e-01,  -8.45e-01],
       [ -9.18e-02,  -2.43e+00],
       [ -9.99e-01,  -4.59e-01],
       [ -5.34e+00,  -4.81e-03]])

In [362]:
np.exp(ps[0:20])

array([[ 0.31,  0.69],
       [ 0.98,  0.02],
       [ 0.26,  0.74],
       [ 0.99,  0.01],
       [ 0.95,  0.05],
       [ 0.09,  0.91],
       [ 0.71,  0.29],
       [ 0.44,  0.56],
       [ 0.91,  0.09],
       [ 0.81,  0.19],
       [ 0.88,  0.12],
       [ 0.67,  0.33],
       [ 0.02,  0.98],
       [ 0.54,  0.46],
       [ 0.25,  0.75],
       [ 0.89,  0.11],
       [ 0.57,  0.43],
       [ 0.91,  0.09],
       [ 0.37,  0.63],
       [ 0.  ,  1.  ]])